In [1]:
import pandas as pd

In [2]:
gender_df = pd.read_csv("./Resources/gender_submission.csv")
test_df = pd.read_csv("./Resources/test.csv")
train_df = pd.read_csv("./Resources/train.csv")

In [3]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train_df[["Survived", "Sex", "Age", "Fare"]].groupby("Sex").mean()

,Survived,Age,Fare
Sex,,,
female,0.742038,27.915709,44.479818
male,0.188908,30.726645,25.523893


ETL on the Empress of Ireland

In [12]:
ireland_df = pd.read_csv("./Resources/empress_of_ireland_passenger_list.csv")

In [55]:
# This Dataset was made by hand so its bound to have some missing or incorrect data in it
# Since I made this dataset with this project in mind, there isn't a lot of ETL to be done on it (other than when I was typing it up)

ireland_df.count()

# The counts appear to be little off

Survived    1469
Pclass      1469
Name        1469
Sex         1468
Age          809
dtype: int64

In [14]:
ireland_df.columns

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age'], dtype='object')

In [41]:
# I want to check that all the categories are correct
survived_unique = ireland_df["Survived"].unique()
pclass_unique = ireland_df["Pclass"].unique()
sex_unique = ireland_df["Sex"].unique()

print(f"Unique values for 'Survived' (this should be 1,0): {survived_unique}")
print(f"Unique values for 'Pclass' (this should be 0,1,2,3): {pclass_unique}")
print(f"Unique values for 'Sex' (this should be male/female/nan): {sex_unique}")

# Looks like they are

Unique values for 'Survived' (this should be 1,0): [1 0]
Unique values for 'Pclass' (this should be 0,1,2,3): [0 1 2 3]
Unique values for 'Sex' (this should be male/female/nan): ['male' 'female' nan]


In [15]:
ireland_df.head()

,Survived,Pclass,Name,Sex,Age
0,1,0,"Kendall, Henry George",male,NaN
1,0,0,"Steede, Mansfield Richard",male,NaN
2,1,0,"Jones, Edward John",male,NaN
3,0,0,"Williams, Roger",male,NaN
4,1,0,"File, Osmond",male,NaN


In [19]:
# The expected numbers are taken from wikipedia, I've gone over the passenger and crew list that I had and couldn't find anything missing
# so either Wikipedia or the list I was looking at was wrong

ireland_df.loc[ireland_df["Pclass"] == 0].count()
#Missing 3 crew members (expected 420)

Survived    417
Pclass      417
Name        417
Sex         417
Age           0
dtype: int64

In [20]:
ireland_df.loc[ireland_df["Pclass"] == 1].count()
# I have 1 extra 1st class passenger (expected 87)

Survived    88
Pclass      88
Name        88
Sex         88
Age         87
dtype: int64

In [21]:
ireland_df.loc[ireland_df["Pclass"] == 2].count()
# Missing 1 2nd class passenger (expected 253)

Survived    252
Pclass      252
Name        252
Sex         252
Age         234
dtype: int64

In [22]:
# Third class passengers I haven't double checked yet, its a lot of data to go through by hand, and I think this is close enough

ireland_df.loc[ireland_df["Pclass"] == 3].count()
# Missing 5 3rd class pasengers (expected 717)

Survived    712
Pclass      712
Name        712
Sex         711
Age         488
dtype: int64

In [23]:
ireland_df.groupby(["Pclass"]).mean()

,Survived,Age
Pclass,,
0,0.558753,NaN
1,0.375000,39.080460
2,0.194444,30.924786
3,0.185393,29.055266


In [31]:
# Average Age of the survivors

ireland_df.loc[ireland_df["Survived"] == 1].groupby("Pclass").mean()["Age"]

Pclass
0          NaN
1    35.500000
2    31.156250
3    32.440367
Name: Age, dtype: float64

In [32]:
# Average age of those who died

ireland_df.loc[ireland_df["Survived"] == 0].groupby("Pclass").mean()["Age"]

Pclass
0          NaN
1    41.163636
2    30.865054
3    28.081715
Name: Age, dtype: float64

In [26]:
ireland_df.dtypes


Survived      int64
Pclass        int64
Name         object
Sex          object
Age         float64
dtype: object

Now for ETL on the estonia passenger list

In [44]:
estonia_df = pd.read_csv("./Resources/estonia-passenger-list.csv")
estonia_df.head()

,PassengerId,Country,Firstname,Lastname,Sex,Age,Category,Survived
0,1,Sweden,ARVID KALLE,AADLI,M,62,P,0
1,2,Estonia,LEA,AALISTE,F,22,C,0
2,3,Estonia,AIRI,AAVASTE,F,21,C,0
3,4,Sweden,JURI,AAVIK,M,53,C,0
4,5,Sweden,BRITTA ELISABET,AHLSTROM,F,55,P,0


In [63]:
# Format the names like they are in the Titanic dataset (I don't think we're going to add Mr. or Mrs. like they are in the Titanic and Empress of Ireland)
estonia_df["Name"] = estonia_df["Lastname"].str.title() + ', ' + estonia_df["Firstname"].str.title()

# Change how gender is displayed
estonia_df.loc[estonia_df["Sex"] == "M", "Sex"] = "male"
estonia_df.loc[estonia_df["Sex"] == "F", "Sex"] = "female"

# Change the column called Category to Pclass that only contains 1 (for passenger) and 0 (for crew) to match the Empress of Ireland dataset

estonia_df.rename(columns = {"Category":"Pclass"}, inplace=True)

estonia_df.loc[estonia_df["Pclass"] == "P", "Pclass"] = 1
estonia_df.loc[estonia_df["Pclass"] == "C", "Pclass"] = 0

estonia_df = estonia_df.astype({"Pclass": int})

estonia_df.head()

,PassengerId,Country,Firstname,Lastname,Sex,Age,Pclass,Survived,Name
0,1,Sweden,ARVID KALLE,AADLI,male,62,1,0,"Aadli, Arvid Kalle"
1,2,Estonia,LEA,AALISTE,female,22,0,0,"Aaliste, Lea"
2,3,Estonia,AIRI,AAVASTE,female,21,0,0,"Aavaste, Airi"
3,4,Sweden,JURI,AAVIK,male,53,0,0,"Aavik, Juri"
4,5,Sweden,BRITTA ELISABET,AHLSTROM,female,55,1,0,"Ahlstrom, Britta Elisabet"


In [71]:
new_estonia_df = estonia_df[["Survived", "Pclass", "Name", "Sex", "Age"]]
new_estonia_df.head()

,Survived,Pclass,Name,Sex,Age
0,0,1,"Aadli, Arvid Kalle",male,62
1,0,0,"Aaliste, Lea",female,22
2,0,0,"Aavaste, Airi",female,21
3,0,0,"Aavik, Juri",male,53
4,0,1,"Ahlstrom, Britta Elisabet",female,55


In [72]:
new_estonia_df.dtypes

Survived     int64
Pclass       int64
Name        object
Sex         object
Age          int64
dtype: object

In [75]:
new_estonia_df["Survived"].unique()

array([0, 1])

In [56]:
# Now save the cleaned dataframe

new_estonia_df.to_csv("./Resources/estonia-passenger-list-cleaned.csv", index=False)